In [12]:
import requests
import pandas as pd

# Read in the cities and their zip codes
CityZipCodes = pd.read_csv("cityzipcodes.csv",sep="|")
CityZipCodes = CityZipCodes.loc[:,"City":"Zipcode"]
CityZipCodes.head()

,City,State,Zipcode
0,Aberdeen,WA,98520
1,Aberdeen,WA,98535
2,Aberdeen,WA,98536
3,Aberdeen,WA,98537
4,Aberdeen,WA,98547


In [16]:
CaliforniaCities = CityZipCodes.loc[CityZipCodes['State'] == "CA"]
CaliforniaCities.head()

,City,State,Zipcode
392,Antelope Vly,CA,93534
393,Antelope Vly,CA,93535
394,Antelope Vly,CA,93536
395,Antelope Vly,CA,93539
396,Antelope Vly,CA,93543


In [30]:
#http://www.airnowapi.org/aq/observation/zipCode/historical/?format=application/json&zipCode=20002&date=2018-11-28T00-0000&distance=25&API_KEY=3F8A6B70-6908-4FB0-92F8-10D3FC3176E5
url = "http://www.airnowapi.org/aq/observation/zipCode/historical/"
querystring = {"format":"csv","zipCode":"20002","date":"2018-11-28T00-0000","distance":"25","API_KEY":"3F8A6B70-6908-4FB0-92F8-10D3FC3176E5"}
response = requests.request("GET", url, params=querystring)
print(response.text)
cr = csv.reader(response)

"DateObserved","HourObserved","LocalTimeZone","ReportingArea","StateCode","Latitude","Longitude","ParameterName","AQI","CategoryNumber","CategoryName"
"2018-11-28 ","0","EST","Metropolitan Washington","DC","38.919","-77.013","OZONE","24","1","Good"
"2018-11-28 ","0","EST","Metropolitan Washington","DC","38.919","-77.013","PM2.5","16","1","Good"
"2018-11-28 ","0","EST","Metropolitan Washington","DC","38.919","-77.013","PM10","15","1","Good"



In [46]:
# Create a map from AQSID to county name for all counties in California
SiteLocations = pd.read_table("monitoring_site_locations.dat",sep="|",encoding = "ISO-8859-1")
CASiteLocations = SiteLocations.loc[SiteLocations['state name'] == "CA"]
CASiteLocations = CASiteLocations.loc[:,['AQSID','state name','county name']].drop_duplicates()
CASiteLocations

,AQSID,state name,county name
413,060271023,CA,INYO
463,060731010,CA,SAN DIEGO
550,060431003,CA,MARIPOSA
605,060798006,CA,SAN LUIS OBISPO
664,060295002,CA,KERN
738,060550004,CA,NAPA
777,840060299999,CA,KERN
1268,060816001,CA,SAN MATEO
1274,060976001,CA,SONOMA
1299,060856001,CA,SANTA CLARA


In [91]:
# Create a county name data frame
CountyAirQuality = CASiteLocations.loc[:,'county name'].to_frame().drop_duplicates().sort_values(by=['county name'])
CountyAirQuality['Good (days)'] = 0
CountyAirQuality['Moderate (days)'] = 0
CountyAirQuality['Unhealthy for Sensitive Groups (days)'] = 0
CountyAirQuality['Unhealthy (days)'] = 0
CountyAirQuality['Very Unhealthy (days)'] = 0
CountyAirQuality['Hazardous (days)'] = 0
CountyAirQuality['Total (days)'] = 0

CountyAirQuality

,county name,Good (days),Moderate (days),Unhealthy for Sensitive Groups (days),Unhealthy (days),Very Unhealthy (days),Hazardous (days),Total (days)
1726,ALAMEDA,0,0,0,0,0,0,0
1869,AMADOR,0,0,0,0,0,0,0
1476,BUTTE,0,0,0,0,0,0,0
4233,CALAVERAS,0,0,0,0,0,0,0
4043,COLUSA,0,0,0,0,0,0,0
1407,CONTRA COSTA,0,0,0,0,0,0,0
3482,EL DORADO,0,0,0,0,0,0,0
1758,FRESNO,0,0,0,0,0,0,0
4101,GLENN,0,0,0,0,0,0,0
1360,HUMBOLDT,0,0,0,0,0,0,0


In [92]:
# Experiment with data @20171231
file_path = "./2018/20171231-peak.dat"
AirQualityDataFrame = pd.read_csv(file_path,names=['Date','AQSID','sitename','parameter name','reporting units','value','averaging period','data source'],sep="|",encoding = "ISO-8859-1")
# Only care about PM2.5 for now
PM25DataFrame = AirQualityDataFrame.loc[AirQualityDataFrame['parameter name'] == 'PM2.5-24hr',:]
#print(PM25DataFrame)
for index, row in PM25DataFrame.iterrows() :
    CurrentAQSID = row['AQSID']
    CurrentValue = row['value']
    CountyNameSeries = CASiteLocations.loc[CASiteLocations['AQSID'] == CurrentAQSID,:]
    if CountyNameSeries['county name'].count() != 0 :
        CurrentCountyName = CountyNameSeries.iloc[0]['county name']
        CountyAirQuality.loc[CountyAirQuality['county name'] == CurrentCountyName, 'Total (days)'] += 1
        if CurrentValue <= 50 : 
            CountyAirQuality.loc[CountyAirQuality['county name'] == CurrentCountyName, 'Good (days)'] += 1
        elif CurrentValue <= 100 :
            CountyAirQuality.loc[CountyAirQuality['county name'] == CurrentCountyName, 'Moderate (days)'] += 1
        elif CurrentValue <= 150 :
            CountyAirQuality.loc[CountyAirQuality['county name'] == CurrentCountyName, 'Unhealthy for Sensitive Groups (days)'] += 1
        elif CurrentValue <= 200 :
            CountyAirQuality.loc[CountyAirQuality['county name'] == CurrentCountyName, 'Unhealthy (days)'] += 1
        elif CurrentValue <= 300 :
            CountyAirQuality.loc[CountyAirQuality['county name'] == CurrentCountyName, 'Very Unhealthy (days)'] += 1
        elif CurrentValue <= 500 :
            CountyAirQuality.loc[CountyAirQuality['county name'] == CurrentCountyName, 'Hazardous (days)'] += 1

CountyAirQuality

,county name,Good (days),Moderate (days),Unhealthy for Sensitive Groups (days),Unhealthy (days),Very Unhealthy (days),Hazardous (days),Total (days)
1726,ALAMEDA,5,0,0,0,0,0,5
1869,AMADOR,0,0,0,0,0,0,0
1476,BUTTE,3,0,0,0,0,0,3
4233,CALAVERAS,1,0,0,0,0,0,1
4043,COLUSA,1,0,0,0,0,0,1
1407,CONTRA COSTA,2,0,0,0,0,0,2
3482,EL DORADO,0,0,0,0,0,0,0
1758,FRESNO,0,3,0,0,0,0,3
4101,GLENN,1,0,0,0,0,0,1
1360,HUMBOLDT,0,0,0,0,0,0,0


In [93]:
# Now let's do this for 20180101 to 20181120
DateList = [d.strftime('%Y%m%d') for d in pd.date_range('1/1/2018','11/20/2018')]
for d in DateList :
    file_path = "./2018/" + d + "-peak.dat"
    AirQualityDataFrame = pd.read_csv(file_path,names=['Date','AQSID','sitename','parameter name','reporting units','value','averaging period','data source'],sep="|",encoding = "ISO-8859-1")
    # Only care about PM2.5 for now
    PM25DataFrame = AirQualityDataFrame.loc[AirQualityDataFrame['parameter name'] == 'PM2.5-24hr',:]
    print("Processing date:", d)
    for index, row in PM25DataFrame.iterrows() :
        CurrentAQSID = row['AQSID']
        CurrentValue = row['value']
        CountyNameSeries = CASiteLocations.loc[CASiteLocations['AQSID'] == CurrentAQSID,:]
        if CountyNameSeries['county name'].count() != 0 :
            CurrentCountyName = CountyNameSeries.iloc[0]['county name']
            CountyAirQuality.loc[CountyAirQuality['county name'] == CurrentCountyName, 'Total (days)'] += 1
            if CurrentValue <= 50 : 
                CountyAirQuality.loc[CountyAirQuality['county name'] == CurrentCountyName, 'Good (days)'] += 1
            elif CurrentValue <= 100 :
                CountyAirQuality.loc[CountyAirQuality['county name'] == CurrentCountyName, 'Moderate (days)'] += 1
            elif CurrentValue <= 150 :
                CountyAirQuality.loc[CountyAirQuality['county name'] == CurrentCountyName, 'Unhealthy for Sensitive Groups (days)'] += 1
            elif CurrentValue <= 200 :
                CountyAirQuality.loc[CountyAirQuality['county name'] == CurrentCountyName, 'Unhealthy (days)'] += 1
            elif CurrentValue <= 300 :
                CountyAirQuality.loc[CountyAirQuality['county name'] == CurrentCountyName, 'Very Unhealthy (days)'] += 1
            elif CurrentValue <= 500 :
                CountyAirQuality.loc[CountyAirQuality['county name'] == CurrentCountyName, 'Hazardous (days)'] += 1
    

Processing date: 20180101
Processing date: 20180102
Processing date: 20180103
Processing date: 20180104
Processing date: 20180105
Processing date: 20180106
Processing date: 20180107
Processing date: 20180108
Processing date: 20180109
Processing date: 20180110
Processing date: 20180111
Processing date: 20180112
Processing date: 20180113
Processing date: 20180114
Processing date: 20180115
Processing date: 20180116
Processing date: 20180117
Processing date: 20180118
Processing date: 20180119
Processing date: 20180120
Processing date: 20180121
Processing date: 20180122
Processing date: 20180123
Processing date: 20180124
Processing date: 20180125
Processing date: 20180126
Processing date: 20180127
Processing date: 20180128
Processing date: 20180129
Processing date: 20180130
Processing date: 20180131
Processing date: 20180201
Processing date: 20180202
Processing date: 20180203
Processing date: 20180204
Processing date: 20180205
Processing date: 20180206
Processing date: 20180207
Processing d

Processing date: 20181113
Processing date: 20181114
Processing date: 20181115
Processing date: 20181116
Processing date: 20181117
Processing date: 20181118
Processing date: 20181119
Processing date: 20181120


In [94]:
CountyAirQuality

,county name,Good (days),Moderate (days),Unhealthy for Sensitive Groups (days),Unhealthy (days),Very Unhealthy (days),Hazardous (days),Total (days)
1726,ALAMEDA,1682,47,15,8,0,0,1752
1869,AMADOR,0,0,0,0,0,0,0
1476,BUTTE,911,21,4,4,6,2,948
4233,CALAVERAS,311,5,0,0,0,0,316
4043,COLUSA,508,22,5,0,1,0,536
1407,CONTRA COSTA,593,14,5,2,0,0,614
3482,EL DORADO,0,0,0,0,0,0,0
1758,FRESNO,932,28,0,0,0,0,960
4101,GLENN,293,8,0,1,0,0,302
1360,HUMBOLDT,94,1,0,0,0,0,95
